In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms

In [18]:
class BetterCNN(nn.Module):
    def __init__(self, in_channels, classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, (3,3))
        self.conv2 = nn.Conv2d(32, 32, (3,3))
        self.conv3 = nn.Conv2d(32, 64, (3,3))
        self.conv4 = nn.Conv2d(64, 64, (3,3))
        self.mp = nn.MaxPool2d((2,2))
        self.flatten = nn.Flatten(1)

        self.l1 = nn.Linear(1600, 512)
        self.l2 = nn.Linear(512, 128)
        self.l3 = nn.Linear(128, classes)
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.mp(self.conv2(x)))
        x = self.relu(self.conv3(x))
        x = self.relu(self.mp(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        out = torch.softmax(self.l3(x), 1)
        
        """
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        """

        return out

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./dataset', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./dataset', train=False, download=True, transform=transform)

train_dataloader = DataLoader(trainset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(testset, batch_size=128, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
model = BetterCNN(3, 10)
optim = torch.optim.AdamW(net.parameters())
loss_fn = nn.CrossEntropyLoss()

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    losses = []
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            losses.append(loss)
    return losses

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct * 100

In [ ]:
epochs = 20
training_losses = []
test_accuracy = []
for t in range(epochs):
    print(f"Epoch {t+1} - ", end="")
    train_losses = train(train_dataloader, model, loss_fn, optim)
    test_acc = test(test_dataloader, model, loss_fn)
    training_losses += train_losses
    test_accuracy.append(test_acc)

Epoch 1 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302620 

Epoch 2 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302603 

Epoch 3 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302620 

Epoch 4 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302606 

Epoch 5 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302620 

Epoch 6 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302583 

Epoch 7 - Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302586 

Epoch 8 - 